# Assignment 2
**Name: Rommel Antunez Barrios**

**e-mail: rommel.antunez6474@alumnos.udg.mx**

# MODULES


In [1]:
import numpy as np
from scipy.stats import wrapcauchy
from matplotlib import pyplot as plt
from scipy.spatial import distance
import math

from scipy.stats import levy_stable

# from plotly
import plotly.graph_objects as go

# from pandas
import pandas as pd

# CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Activity 1: Path length - (BM1 vs BM2 vs CRW)

In [3]:
# Write a function that returns a Brownian motion (BM) trajectory in pandas df
def brownian_motion(speed=3, n_steps=1000, s_pos=[0,0]):
    # Init velocity vector
    velocity = Vec2d(speed,0)

    # Init empty DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])
    # Add starting position to DF
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True) # Pandas tienes nombre de columnas pero tambien filas, llamadas index
    # Generate trajectory
    for i in range(n_steps-1):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        # We are rotating the velocity vector
        velocity = velocity.rotated(turn_angle)

        # Update to new position
        temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i] + velocity.x, 'y_pos':BM_2d_df.y_pos[i] + velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True) # Pandas tienes nombre de columnas pero tambien filas, llamadas index
    return BM_2d_df
# Write a function that returns a Correlated Random Walk (CRW) trajectory in pandas df
def correlated_random_walk(speed=3,  CRW_coefficient=0.5, n_steps=1000, s_pos=0):
    # Init velocity vector
    velocity = Vec2d(speed,0) # Implement a Vec2d class vector

    # Matrix for the Brownian Walker
    CRW = np.ones(shape=(n_steps, 2)) * s_pos # np array for storing the trajectory
    rotations = wrapcauchy.rvs(c=CRW_coefficient, loc=0,size=n_steps)  # Selecting rotation from cauchy distribution
    for i in range(1, n_steps):
        turn_angle = np.random.choice(rotations)
        # We are rotating the velocity vector
        velocity = velocity.rotated(turn_angle)
        # Update to new position
        CRW[i,0] = CRW[i-1,0] + velocity.x
        CRW[i,1] = CRW[i-1,1] + velocity.y
    CRW_df = pd.DataFrame(CRW, columns=['x_pos','y_pos'])
    return CRW_df

# Write a function that returns the path length of given trajectory
def path_length(trajectory):
    # Init variables
    path_length = 0
    pl_trajectory = pd.DataFrame(columns=['x_pos','y_pos'])
    # Calculate path length
    for i in range(len(trajectory)-1):
        path_length += distance.euclidean([trajectory.x_pos[i], trajectory.y_pos[i]], [trajectory.x_pos[i+1], trajectory.y_pos[i+1]])
        temp_df = pd.DataFrame([{'x_pos': i, 'y_pos':path_length}])
        pl_trajectory = pd.concat([pl_trajectory, temp_df], ignore_index=True) 
    return pl_trajectory

# Compare at least the path length of three trajectories

# Display the results using plotly
fig = go.Figure()
trajectory = path_length(brownian_motion(speed=3))
fig.add_trace(go.Scatter
(
    x=trajectory.x_pos,
    y=trajectory.y_pos,
    mode='lines+markers',
    name='Brownian motion (speed=3)',
    line=dict(color='red', width=5)
))
trajectory = path_length(brownian_motion(speed=6))
fig.add_trace(go.Scatter
(
    x=trajectory.x_pos,
    y=trajectory.y_pos,
    mode='lines+markers',
    name='Brownian motion (speed=6)',
    line=dict(color='green', width=3)
))
trajectory = path_length(correlated_random_walk(speed=3, CRW_coefficient=0.5))
fig.add_trace(go.Scatter
(
    x=trajectory.x_pos,
    y=trajectory.y_pos,
    name='CRW (speed=3, CRW_coefficient=0.5)',
    line=dict(color='purple', width=2)
))
fig.show()

C:\Users\Servicio_Social\AppData\Local\Temp\ipykernel_14944\608963143.py:49: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Servicio_Social\AppData\Local\Temp\ipykernel_14944\608963143.py:49: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Servicio_Social\AppData\Local\Temp\ipykernel_14944\608963143.py:49: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when

# Activity 2: Mean Squared Displacement - (BM vs CRW)

In [4]:
def mean_squared_displacement(trajectory):
    msd = []
    for n in range(1, len(trajectory)):
        difference = np.array([distance.euclidean(trajectory.iloc[i-n], trajectory.iloc[i]) for i in range(n, trajectory.shape[0], 1)])
        difference = difference ** 2
        difference = np.mean(difference)
        msd.append(difference)
    return pd.DataFrame(msd, columns=['msd'])

# Calculate MSD for the correlated random walk
crw_trajectory = correlated_random_walk(speed=6, CRW_coefficient=0.9)
msd_crw = mean_squared_displacement(crw_trajectory)
bm_trajectory = brownian_motion(speed=6)
msd_bm = mean_squared_displacement(bm_trajectory)

# Plot MSD using Plotly
fig_msd = go.Figure()
fig_msd.add_trace(go.Scatter(
    x=msd_crw.index,
    y=msd_crw['msd'],
    mode='lines+markers',
    name='MSD (CRW, speed=6, CRW_coefficient=0.9)',
    line=dict(color='blue', width=2)
))
fig_msd.add_trace(go.Scatter(
    x=msd_bm.index,
    y=msd_bm['msd'],
    mode='lines+markers',
    name='MSD (BM, speed=6)',
    line=dict(color='red', width=2)
))
fig_msd.update_layout(title='Mean Squared Displacement')
                      
fig_msd.show()


# Activity 3: Turning-angle Distribution - (source dist. vs observed dist.)

In [5]:
# Consider two CRW trajectories with different Cauchy coefficients.
crw_trajectory_0_6 = correlated_random_walk(speed=6, CRW_coefficient=0.6)
crw_trajectory_0_9 = correlated_random_walk(speed=6, CRW_coefficient=0.9)

# Write a function that returns the turning angles for a given trajectory.
def turning_angles(trajectory):
    turning_angles = []
    for i in range(1, len(trajectory)-1):
        # Calculate the turning angle
        a = np.array([trajectory.x_pos[i-1], trajectory.y_pos[i-1]])
        b = np.array([trajectory.x_pos[i], trajectory.y_pos[i]])
        c = np.array([trajectory.x_pos[i+1], trajectory.y_pos[i+1]])
        ab = b - a
        bc = c - b
        cross_product = np.cross(ab, bc)
        orient = np.sign(cross_product)
        if orient == 0:
            orient = 1
        denominator = np.linalg.norm(ab) * np.linalg.norm(bc) + np.finfo(float).eps
        angle = np.arcsin(np.linalg.norm(cross_product) / denominator) # apply the arcsin function to the dot product to get the angle without cos 
        new_angle = angle * orient
        turning_angles.append(round(new_angle, 2))
    return pd.DataFrame(turning_angles, columns=['turning_angle'])

# Calculate turning angles for both trajectories

turning_angles_0_6 = turning_angles(crw_trajectory_0_6)
turning_angles_0_9 = turning_angles(crw_trajectory_0_9)

# Get the curve for the Cauchy distribution
# Cauchy distribution for CRW with coefficient 0.6
x_values_0_6 = np.linspace(min(turning_angles_0_6.turning_angle), max(turning_angles_0_6.turning_angle), turning_angles_0_6.shape[0])
y_values_0_6 = np.array([wrapcauchy.pdf(abs(i), 0.6) for i in x_values_0_6]) * 100
# I use scalar multiplication to increase the size of the curve

# Cauchy distribution for CRW with coefficient 0.6
x_values_0_9 = np.linspace(min(turning_angles_0_9.turning_angle), max(turning_angles_0_9.turning_angle), turning_angles_0_9.shape[0])
y_values_0_9 = np.array([wrapcauchy.pdf(abs(i), 0.9) for i in x_values_0_9]) * 100
# I use scalar multiplication to increase the size of the curve

# Plot histograms using Plotly
fig_hist = go.Figure()
# Plot curve cauchy distribution 0.6
fig_hist.add_trace(go.Scatter(
    x=x_values_0_6,
    y=y_values_0_6,
    mode='lines',
    name='cauchy 0.6',
    line=dict(color='blue', width=2)
)) 
# Plot curve cauchy distribution 0.9
fig_hist.add_trace(go.Scatter(
    x=x_values_0_9,
    y=y_values_0_9,
    mode='lines',
    name='cauchy 0.9',
    line=dict(color='red', width=2)
)) 

# Plot histogram for turning angles 0.6
fig_hist.add_trace(go.Histogram(
    x=turning_angles_0_6.turning_angle,
    name='CRW (speed=6, CRW_coefficient=0.6)',
    marker_color='blue'
))

# Plot histogram for turning angles 0.9
fig_hist.add_trace(go.Histogram(
    x=turning_angles_0_9.turning_angle,
    name='CRW (speed=6, CRW_coefficient=0.9)',
    marker_color='red'
))
fig_hist.update_layout(title='Turning Angles Histogram and Curve')
fig_hist.show()

C:\Users\Servicio_Social\AppData\Local\Temp\ipykernel_14944\4242030742.py:15: DeprecationWarning:

Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)



# Activity 4: Step-length Distribution - (source dist. vs observed dist.)

In [39]:
# Write a function that returns a Lévy Walk (LW) trajectory in pandas df.
def levy_walk(alpha=0.9, beta=1,  loc=0, n_steps=1000, s_pos=[0,0,0]):
    speed = 6
    time_per_step = 0.001
    # Init empty DF
    LW_df = pd.DataFrame(columns=['x_pos','y_pos','z_pos'])
    # Add starting position to DF
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1], 'z_pos':s_pos[1]}])
    LW_df = pd.concat([LW_df, temp_df], ignore_index=True)
    # Initialize the velocity vector with initial position
    velocity = Vec2d(speed, 0)
    #angle rotation values
    step_length = rotating_angles = levy_stable.rvs(alpha, beta, loc=loc, size=n_steps) 
    # Random values using Lévy algorithm
    for i in range(n_steps):
        # Rotation angles for the vector are also random values from lévy distribution
        velocity = velocity.rotated(rotating_angles[i])
        
        length_traveled = velocity.get_length() * step_length * time_per_step # To get z position
        
        # new position values are stored in the trajectory dataframe
        temp_df = pd.DataFrame([{
            'x_pos':LW_df.x_pos[i] + velocity.x * step_length, 
            'y_pos':LW_df.y_pos[i] + velocity.y * step_length, 
            'z_pos':LW_df.z_pos[i] + length_traveled
            }]
        )
        LW_df = pd.concat([LW_df, temp_df], ignore_index=True)
        
    return LW_df, rotating_angles

# Generate trajectory
# Consider two LW trajectories with different alpha coefficients.
# trajectory 1
levy_trajectory_1, rotating_angles_1 = levy_walk(alpha=1.0, beta=1, loc=0, n_steps=1000)

#trajectory 2
levy_trajectory_2, rotating_angles_2 = levy_walk(alpha=0.7, beta=1, loc=0, n_steps=1000)

# Write a function that returns the step lengths for a given trajectory.
def step_lengths(trajectory, rotating_angles):
    accumulated_step_length = []
    step_length = 1 # Initialize step length
    for i in range(len(trajectory)-1):
        if (abs(rotating_angles[i] < 10 ) ):
            step_length = step_length + 1
        else:
            accumulated_step_length.append(step_length)
            step_length = 1

    return pd.DataFrame(accumulated_step_length, columns=['step_length'])
# Compare the observed distribution (histogram) to the source distribution (curve) for both trajectories.
accumulated_step_lengths_1 = step_lengths(levy_trajectory_1, rotating_angles_1)
accumulated_step_lengths_2 = step_lengths(levy_trajectory_2, rotating_angles_2)

# Display the results using plotly.
x1 = np.linspace(0,50, len(accumulated_step_lengths_1))
x2 = np.linspace(0,50, len(accumulated_step_lengths_2))

levy_pdf_1 = levy_stable.pdf(x1, alpha=1.0, beta=1,loc=0)
levy_pdf_2 = levy_stable.pdf(x2, alpha=0.7, beta=1,loc=0)

# Scalar factor
scaling_factor_1 = 200
levy_pdf_1 *= scaling_factor_1
scaling_factor_2 = 200
levy_pdf_2 *= scaling_factor_2

# Plot histograms using Plotly
fig = go.Figure()
fig.add_trace(go.Histogram(
    y=x1,
    x=accumulated_step_lengths_1.step_length,
    name='Levy - Alpha 1.0 Beta 1.0 observable',
    marker_color='blue'
))
fig.add_trace(go.Scatter(
    x = x1,
    y = levy_pdf_1,
    mode = 'lines',
    name='Levy - Alpha 1.0 Beta 1.0 source',
    showlegend=True
))

fig.add_trace(go.Histogram(
    y=x2,
    x=accumulated_step_lengths_2.step_length,
    name='Levy - Alpha 0.7 Beta 1.0 observable',
    marker_color='green'
))
fig.add_trace(go.Scatter(
    x = x2,
    y = levy_pdf_2,
    mode = 'lines',
    name='Levy - Alpha 0.7 Beta 1.0 source',
    line=dict(color='purple', width=3),
    showlegend=True
))
fig.update_layout(title='Step Lengths Histogram and Curve')
fig.show()